In [9]:
import os
import re
import pandas as pd
import spacy
from spacy import displacy
nlp = spacy.load('en')

In [37]:
data_dir = 'mitford-letters'
clean_text_dir = '../mitford_letters_clean'
gold_standard_dir = '../mitford_letters_gs'
spacy_dir = '../spacy_mitford'
letters = os.listdir(clean_text_dir)

In [41]:
for letter in letters:
    with open(os.path.join(clean_text_dir, letter), 'r') as f:
        text = f.read()
    
    text = text.strip()
    text = re.sub('\n', '', text)
    text = re.sub('\t', '', text)
    doc = nlp(text)
    ents = doc.ents
    ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents if e.label_ in ['PERSON', 'GPE']]
    
    tags = []
    begin_offsets = []
    end_offsets = []
    text = []
    for e in ents:
        text.append(e[0])
        tags.append(e[-1])
        begin_offsets.append(e[1])
        end_offsets.append(e[2])
        
    spacy_df = pd.DataFrame({'tag': tags,
                            'begin_offsets': begin_offsets,
                            'end_offset': end_offsets,
                            'text': text})
    spacy_df['tag'].loc[spacy_df['tag'] == 'GPE'] = 'place'
    spacy_df['tag'].loc[spacy_df['tag'] == 'PERSON'] = 'person'
    spacy_df.to_csv(os.path.join(spacy_dir, letter), index=False)

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [40]:
gs = pd.read_csv(os.path.join(gold_standard_dir, letter))
gs.sort_values(by=['begin_offset'], inplace=True)
gs

,tag,begin_offset,end_offset,text,ref
0,person,6,22,B.R. Haydon Esqr,#Haydon
1,person,1954,1965,Mrs. Haydon,#Haydon_Mrs
2,person,2308,2322,M. R. Mitford.,#MRM
3,person,2326,2344,B. R. Haydon Esqre,#Haydon
4,place,2364,2383,King's Bench Prison,#Kings_Bench_Prison
5,place,2386,2392,London,#London_city
